In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

ModuleNotFoundError: No module named 'selenium'

In [1]:
# pip install virtualenv

  Obtaining dependency information for virtualenv from https://files.pythonhosted.org/packages/07/4d/410156100224c5e2f0011d435e477b57aed9576fc7fe137abcf14ec16e11/virtualenv-20.26.3-py3-none-any.whl.metadata
  Obtaining dependency information for distlib<1,>=0.3.7 from https://files.pythonhosted.org/packages/8e/41/9307e4f5f9976bc8b7fea0b66367734e8faf3ec84bc0d412d8cfabbb66cd/distlib-0.3.8-py2.py3-none-any.whl.metadata
  Obtaining dependency information for filelock<4,>=3.12.2 from https://files.pythonhosted.org/packages/ae/f0/48285f0262fe47103a4a45972ed2f9b93e4c80b8fd609fa98da78b2a5706/filelock-3.15.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
    --------------------------------------- 0.1/5.7 MB 2.0 MB/s eta 0:00:03
   ---- ----------------------------------- 0.7/5.7 MB 8.7 MB/s eta 0:00:01
   ----------- ---------------------------- 1.6/5.7 MB 14.7 MB/s eta 0:00:01
   --------------------- ------------------ 3.0/5.7 MB 19.1 MB/s eta 


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
